In [1]:
import warnings
warnings.filterwarnings('ignore')

import argparse
import time

import numpy as np
from scipy.sparse import coo_matrix, csc_matrix
from scipy.sparse.linalg import svds

import torch
from input_data import load_data
from preprocessing import general_train_test_split_edges,biased_train_test_split_edges, bidirectional_train_test_split_edges
from layers import DirectedGCNConvEncoder, DirectedInnerProductDecoder
from layers import SingleLayerDirectedGCNConvEncoder
from models import DirectedGAE

from layers import InnerProductDecoder
from layers import DummyEncoder, DummyPairEncoder

from models import GAE

In [2]:
parser = argparse.ArgumentParser()

parser.add_argument('--dataset',
                    nargs= '?',
                    default='cora_ml',
                    type=str)
parser.add_argument('--task',
                    nargs= '?',
                    default='task_3',
                    type=str)
parser.add_argument('--model',
                    nargs= '?',
                    default='digae',
                    type=str)
parser.add_argument('--learning_rate',
                    nargs= '?',
                    default=0.01,
                    type=float)
parser.add_argument('--epochs',
                    nargs= '?',
                    default=200,
                    type=int)
parser.add_argument('--hidden',
                    nargs= '?',
                    default=64,
                    type=int)
parser.add_argument('--dimension',
                    nargs= '?',
                    default=32,
                    type=int)
parser.add_argument('--alpha',
                    nargs= '?',
                    default=1.0,
                    type=float)
parser.add_argument('--beta',
                    nargs= '?',
                    default=0.0,
                    type=float)
parser.add_argument('--nb_run',
                    nargs= '?',
                    default=10,
                    type=int)
parser.add_argument('--prop_val',
                    nargs= '?',
                    default=5.0,
                    type=float)
parser.add_argument('--prop_test',
                    nargs= '?',
                    default=10.0,
                    type=float)
parser.add_argument('--verbose',
                    nargs= '?',
                    default=True,
                    type=bool)

parser.add_argument('--self_loops',
                    nargs= '?',
                    default=True,
                    type=bool)
parser.add_argument('--adaptive',
                    nargs= '?',
                    default=False,
                    type=bool)
parser.add_argument('--feature_vector_type',
                    nargs= '?',
                    const=None)
parser.add_argument('--feature_vector_size',
                    nargs= '?',
                    const=None,
                    type=int)
parser.add_argument('--directed',
                    nargs= '?',
                    default=True,
                    type=bool)
parser.add_argument('--logfile',
                    nargs='?',
                    default='logs.json',
                    type=str)
parser.add_argument('--validate',
                    nargs='?',
                    default=False,
                    type=bool)

args,_ = parser.parse_known_args()

In [3]:
def train_single():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    loss.backward()
    optimizer.step()
    return float(loss)


def test_single(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

def dummy_train_single():
    model.train()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    return float(loss)

def dummy_test_single(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)
    

def train_pair():
    model.train()
    optimizer.zero_grad()
    s, t = model.encode(u, v, train_pos_edge_index)
    loss = model.recon_loss(s, t, train_pos_edge_index)
    loss.backward()
    optimizer.step()
    return float(loss)

def test_pair(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        s, t = model.encode(u, v, train_pos_edge_index)
    return model.test(s, t, pos_edge_index, neg_edge_index)

def dummy_train_pair():
    model.train()
    s, t = model.encode(u, v, train_pos_edge_index)
    loss = model.recon_loss(s, t, train_pos_edge_index)
    return float(loss)

def dummy_test_pair(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        s, t = model.encode(u, v, train_pos_edge_index)
    return model.test(s, t, pos_edge_index, neg_edge_index)

def identity_features(data, k=None):
    num_nodes = data.x.size(0)
    x = torch.eye(num_nodes)
    u_hat = x.clone()
    v_hat = x.clone()
    return u_hat, v_hat

In [4]:
dummy_run = False
 
if args.verbose:
    print("Loading data...")

device     = 'cpu'
directed   = args.directed

dataset_name    = args.dataset
out_channels    = args.dimension
hidden_channels = args.hidden

compute_features = identity_features

feature_vector_size = args.feature_vector_size

if args.model in ['gae','source_target']:
    train_func = train_single
    test_func  = test_single
elif args.model in ['dummy']:
    train_func = dummy_train_single
    test_func  = dummy_test_single
elif args.model in ['dummy_pair']:
    train_func = dummy_train_pair
    test_func  = dummy_test_pair
else:
    train_func = train_pair
    test_func  = test_pair

val_ratio  = args.prop_val / 100.
test_ratio = args.prop_test / 100.

loaded_data = load_data(dataset_name, directed=directed)

Loading data...


In [5]:
mean_roc  = []
mean_ap   = []
mean_time = []

for i in range(args.nb_run):
    if args.verbose:
        print("Masking test edges...")

    if args.task == 'task_1':
        data = loaded_data.clone()
        data.train_mask = data.val_mask = data.test_mask = data.y = None
        data            = general_train_test_split_edges(data,
                                                         val_ratio=val_ratio,
                                                         test_ratio=test_ratio,
                                                         directed=directed)
    elif args.task == 'task_2':
        data = loaded_data.clone()
        data.train_mask = data.val_mask = data.test_mask = data.y = None
        data            = biased_train_test_split_edges(data,
                                                        val_ratio=val_ratio,
                                                        test_ratio=test_ratio,
                                                        directed=directed)
    elif args.task == 'task_3':
        data = loaded_data.clone()
        data.train_mask = data.val_mask = data.test_mask = data.y = None
        data            = bidirectional_train_test_split_edges(data,
                                                               val_ratio=val_ratio,
                                                               test_ratio=test_ratio,
                                                               directed=directed)     
    else:
        raise ValueError('Undefined task!')

    data = data.to(device)
    train_pos_edge_index = data.train_pos_edge_index.to(device)
    if args.validate is True:
        test_pos_edge_index  = data.val_pos_edge_index.to(device)
        test_neg_edge_index  = data.val_neg_edge_index.to(device)
    else:
        test_pos_edge_index  = data.test_pos_edge_index.to(device)
        test_neg_edge_index  = data.test_neg_edge_index.to(device)

    in_channels          = data.x.shape[1]

    in_channels = data.x.size(0)
    u, v = compute_features(data, in_channels)
    data.u = u
    data.v = v
    data.x = torch.cat([data.u, data.v], dim=1)
    
    u = data.u.to(device)
    v = data.v.to(device)
    print(u.shape, v.shape)

    x = data.x.to(device)

    if args.model == 'digae':
        encoder = DirectedGCNConvEncoder(in_channels, hidden_channels, out_channels, alpha=args.alpha, beta=args.beta,
                                         self_loops=args.self_loops,
                                         adaptive=args.adaptive)
        decoder = DirectedInnerProductDecoder()
        model   = DirectedGAE(encoder, decoder)
        model   = model.to(device)

    elif args.model == 'digae_single_layer':
        encoder = SingleLayerDirectedGCNConvEncoder(in_channels, out_channels, alpha=args.alpha, beta=args.beta,
                                                    self_loops=args.self_loops,
                                                    adaptive=args.adaptive)
        decoder = DirectedInnerProductDecoder()
        model   = DirectedGAE(encoder, decoder)
        model   = model.to(device)

    elif args.model == 'dummy':
        encoder = DummyEncoder()
        decoder = InnerProductDecoder()
        model   = GAE(encoder, decoder)
        model   = model.to(device)
        dummy_run = True

    elif args.model == 'dummy_pair':
        encoder = DummyPairEncoder()
        decoder = DirectedInnerProductDecoder()
        model   = DirectedGAE(encoder, decoder)
        model   = model.to(device)
        dummy_run =True
    else:
        raise ValueError('Undefined model!')

    optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate, weight_decay=5e-4)

    if args.verbose:
        print("Training...")

    # Flag to compute total running time
    t_start = time.time()
    for epoch in range(args.epochs):
        # Flag to compute running time for each epoch
        t = time.time()
        loss = train_func()
        avg_cost = loss
        print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(avg_cost),
        "time=", "{:.5f}".format(time.time() - t))
    mean_time.append(time.time() - t_start)

    if args.verbose:
        print("Testing model...")
    auc, ap = test_func(test_pos_edge_index, test_neg_edge_index)
    roc_score, ap_score = auc, ap
    mean_roc.append(roc_score)
    mean_ap.append(ap_score)

# if adaptive...
print('=' * 60)
print(args.adaptive)
print(args.model)
print('=' * 60)
    
# Report final results
print("\nTest results for", args.model,
      "model on", args.dataset, "on", args.task, "\n",
      "___________________________________________________\n")

print("AUC scores\n", mean_roc)
print("Mean AUC score: ", np.mean(mean_roc),
      "\nStd of AUC scores: ", np.std(mean_roc), "\n \n")

print("AP scores \n", mean_ap)
print("Mean AP score: ", np.mean(mean_ap),
      "\nStd of AP scores: ", np.std(mean_ap), "\n \n")

print("Running times\n", mean_time)
print("Mean running time: ", np.mean(mean_time),
      "\nStd of running time: ", np.std(mean_time), "\n \n")

Masking test edges...
torch.Size([2995, 2995]) torch.Size([2995, 2995])
Training...
Epoch: 0001 train_loss= 1.38638 time= 0.06102
Epoch: 0002 train_loss= 1.38554 time= 0.09066
Epoch: 0003 train_loss= 1.38415 time= 0.07610
Epoch: 0004 train_loss= 1.38169 time= 0.07998
Epoch: 0005 train_loss= 1.37732 time= 0.08197
Epoch: 0006 train_loss= 1.37019 time= 0.07702
Epoch: 0007 train_loss= 1.35873 time= 0.07469
Epoch: 0008 train_loss= 1.34109 time= 0.08103
Epoch: 0009 train_loss= 1.31538 time= 0.08695
Epoch: 0010 train_loss= 1.28533 time= 0.08100
Epoch: 0011 train_loss= 1.24768 time= 0.09205
Epoch: 0012 train_loss= 1.20997 time= 0.08400
Epoch: 0013 train_loss= 1.18766 time= 0.07801
Epoch: 0014 train_loss= 1.16699 time= 0.08395
Epoch: 0015 train_loss= 1.17852 time= 0.08300
Epoch: 0016 train_loss= 1.12153 time= 0.08300
Epoch: 0017 train_loss= 1.14584 time= 0.09100
Epoch: 0018 train_loss= 1.10259 time= 0.09000
Epoch: 0019 train_loss= 1.12374 time= 0.08500
Epoch: 0020 train_loss= 1.09611 time= 0.08

Epoch: 0179 train_loss= 0.65608 time= 0.08069
Epoch: 0180 train_loss= 0.65743 time= 0.07798
Epoch: 0181 train_loss= 0.66077 time= 0.08095
Epoch: 0182 train_loss= 0.64529 time= 0.08000
Epoch: 0183 train_loss= 0.65293 time= 0.07900
Epoch: 0184 train_loss= 0.65963 time= 0.08901
Epoch: 0185 train_loss= 0.71942 time= 0.07698
Epoch: 0186 train_loss= 0.78124 time= 0.06700
Epoch: 0187 train_loss= 0.91948 time= 0.08000
Epoch: 0188 train_loss= 0.66322 time= 0.07700
Epoch: 0189 train_loss= 0.76470 time= 0.08100
Epoch: 0190 train_loss= 0.86966 time= 0.08300
Epoch: 0191 train_loss= 0.66536 time= 0.07485
Epoch: 0192 train_loss= 0.87744 time= 0.08030
Epoch: 0193 train_loss= 0.68532 time= 0.08300
Epoch: 0194 train_loss= 0.78897 time= 0.07800
Epoch: 0195 train_loss= 0.66397 time= 0.08795
Epoch: 0196 train_loss= 0.77277 time= 0.06623
Epoch: 0197 train_loss= 0.66069 time= 0.08589
Epoch: 0198 train_loss= 0.71893 time= 0.06390
Epoch: 0199 train_loss= 0.67164 time= 0.08611
Epoch: 0200 train_loss= 0.70169 ti

Epoch: 0156 train_loss= 0.64304 time= 0.07507
Epoch: 0157 train_loss= 0.63537 time= 0.07094
Epoch: 0158 train_loss= 0.65665 time= 0.07551
Epoch: 0159 train_loss= 0.65461 time= 0.07194
Epoch: 0160 train_loss= 0.65227 time= 0.07886
Epoch: 0161 train_loss= 0.62533 time= 0.06875
Epoch: 0162 train_loss= 0.64148 time= 0.07611
Epoch: 0163 train_loss= 0.63702 time= 0.07300
Epoch: 0164 train_loss= 0.63462 time= 0.08000
Epoch: 0165 train_loss= 0.62227 time= 0.07542
Epoch: 0166 train_loss= 0.61768 time= 0.07808
Epoch: 0167 train_loss= 0.61449 time= 0.07794
Epoch: 0168 train_loss= 0.61365 time= 0.08106
Epoch: 0169 train_loss= 0.63423 time= 0.07693
Epoch: 0170 train_loss= 0.61725 time= 0.06600
Epoch: 0171 train_loss= 0.59634 time= 0.06955
Epoch: 0172 train_loss= 0.60108 time= 0.07186
Epoch: 0173 train_loss= 0.60410 time= 0.06629
Epoch: 0174 train_loss= 0.59678 time= 0.07695
Epoch: 0175 train_loss= 0.61287 time= 0.06721
Epoch: 0176 train_loss= 0.60795 time= 0.06665
Epoch: 0177 train_loss= 0.62873 ti

Epoch: 0132 train_loss= 0.67831 time= 0.07794
Epoch: 0133 train_loss= 0.72216 time= 0.06754
Epoch: 0134 train_loss= 0.69314 time= 0.06032
Epoch: 0135 train_loss= 0.69764 time= 0.08241
Epoch: 0136 train_loss= 0.70967 time= 0.07464
Epoch: 0137 train_loss= 0.67920 time= 0.07201
Epoch: 0138 train_loss= 0.71259 time= 0.07698
Epoch: 0139 train_loss= 0.68653 time= 0.07302
Epoch: 0140 train_loss= 0.69969 time= 0.07594
Epoch: 0141 train_loss= 0.70105 time= 0.07601
Epoch: 0142 train_loss= 0.70133 time= 0.07585
Epoch: 0143 train_loss= 0.70180 time= 0.07799
Epoch: 0144 train_loss= 0.67763 time= 0.07805
Epoch: 0145 train_loss= 0.69968 time= 0.07605
Epoch: 0146 train_loss= 0.69175 time= 0.07633
Epoch: 0147 train_loss= 0.68240 time= 0.07567
Epoch: 0148 train_loss= 0.68772 time= 0.07701
Epoch: 0149 train_loss= 0.69342 time= 0.07601
Epoch: 0150 train_loss= 0.67983 time= 0.07799
Epoch: 0151 train_loss= 0.69942 time= 0.07594
Epoch: 0152 train_loss= 0.68747 time= 0.07505
Epoch: 0153 train_loss= 0.68630 ti

Epoch: 0108 train_loss= 0.92874 time= 0.07843
Epoch: 0109 train_loss= 0.95889 time= 0.07579
Epoch: 0110 train_loss= 0.91535 time= 0.07277
Epoch: 0111 train_loss= 0.94328 time= 0.07696
Epoch: 0112 train_loss= 0.90414 time= 0.07289
Epoch: 0113 train_loss= 0.93950 time= 0.06600
Epoch: 0114 train_loss= 0.90774 time= 0.07202
Epoch: 0115 train_loss= 0.91847 time= 0.07596
Epoch: 0116 train_loss= 0.90097 time= 0.06389
Epoch: 0117 train_loss= 0.91439 time= 0.06103
Epoch: 0118 train_loss= 0.89993 time= 0.08687
Epoch: 0119 train_loss= 0.91849 time= 0.06304
Epoch: 0120 train_loss= 0.90263 time= 0.07909
Epoch: 0121 train_loss= 0.89400 time= 0.07140
Epoch: 0122 train_loss= 0.89373 time= 0.07602
Epoch: 0123 train_loss= 0.89758 time= 0.07702
Epoch: 0124 train_loss= 0.89335 time= 0.07468
Epoch: 0125 train_loss= 0.89467 time= 0.07402
Epoch: 0126 train_loss= 0.89677 time= 0.07399
Epoch: 0127 train_loss= 0.89436 time= 0.07399
Epoch: 0128 train_loss= 0.88637 time= 0.07401
Epoch: 0129 train_loss= 0.90000 ti

...

Epoch: 0040 train_loss= 0.96192 time= 0.07806
Epoch: 0041 train_loss= 0.95892 time= 0.08908
Epoch: 0042 train_loss= 0.97249 time= 0.07100
Epoch: 0043 train_loss= 0.95600 time= 0.07994
Epoch: 0044 train_loss= 0.95193 time= 0.07700
Epoch: 0045 train_loss= 0.94848 time= 0.07837
Epoch: 0046 train_loss= 0.96073 time= 0.07939
Epoch: 0047 train_loss= 0.95656 time= 0.08093
Epoch: 0048 train_loss= 0.95240 time= 0.07764
Epoch: 0049 train_loss= 0.94246 time= 0.07705
Epoch: 0050 train_loss= 0.93393 time= 0.07430
Epoch: 0051 train_loss= 0.93116 time= 0.06638
Epoch: 0052 train_loss= 0.92911 time= 0.08940
Epoch: 0053 train_loss= 0.95368 time= 0.06083
Epoch: 0054 train_loss= 0.97350 time= 0.07595
Epoch: 0055 train_loss= 0.98681 time= 0.07906
Epoch: 0056 train_loss= 0.99902 time= 0.07698
Epoch: 0057 train_loss= 0.93308 time= 0.07695
Epoch: 0058 train_loss= 0.93782 time= 0.07654
Epoch: 0059 train_loss= 0.97340 time= 0.06900
Epoch: 0060 train_loss= 0.93367 time= 0.07100
Epoch: 0061 train_loss= 0.93631 ti

Epoch: 0016 train_loss= 1.15947 time= 0.08199
Epoch: 0017 train_loss= 1.11501 time= 0.07546
Epoch: 0018 train_loss= 1.13841 time= 0.07829
Epoch: 0019 train_loss= 1.08879 time= 0.07900
Epoch: 0020 train_loss= 1.11183 time= 0.07497
Epoch: 0021 train_loss= 1.07999 time= 0.08045
Epoch: 0022 train_loss= 1.08473 time= 0.08101
Epoch: 0023 train_loss= 1.07422 time= 0.07796
Epoch: 0024 train_loss= 1.05479 time= 0.08242
Epoch: 0025 train_loss= 1.06452 time= 0.07862
Epoch: 0026 train_loss= 1.03661 time= 0.07894
Epoch: 0027 train_loss= 1.05067 time= 0.07855
Epoch: 0028 train_loss= 1.02537 time= 0.07901
Epoch: 0029 train_loss= 1.02883 time= 0.07915
Epoch: 0030 train_loss= 1.01511 time= 0.08099
Epoch: 0031 train_loss= 1.01144 time= 0.07807
Epoch: 0032 train_loss= 0.99110 time= 0.07338
Epoch: 0033 train_loss= 1.00815 time= 0.06876
Epoch: 0034 train_loss= 0.98780 time= 0.08931
Epoch: 0035 train_loss= 0.98615 time= 0.07100
Epoch: 0036 train_loss= 0.97129 time= 0.07500
Epoch: 0037 train_loss= 0.96579 ti

Epoch: 0195 train_loss= 0.56235 time= 0.07605
Epoch: 0196 train_loss= 0.54365 time= 0.07895
Epoch: 0197 train_loss= 0.54429 time= 0.07233
Epoch: 0198 train_loss= 0.54980 time= 0.07552
Epoch: 0199 train_loss= 0.55758 time= 0.06676
Epoch: 0200 train_loss= 0.55783 time= 0.07794
Testing model...
Masking test edges...
torch.Size([2995, 2995]) torch.Size([2995, 2995])
Training...
Epoch: 0001 train_loss= 1.38681 time= 0.07811
Epoch: 0002 train_loss= 1.38544 time= 0.07889
Epoch: 0003 train_loss= 1.38399 time= 0.08004
Epoch: 0004 train_loss= 1.38119 time= 0.07797
Epoch: 0005 train_loss= 1.37710 time= 0.07704
Epoch: 0006 train_loss= 1.37054 time= 0.07800
Epoch: 0007 train_loss= 1.36061 time= 0.06995
Epoch: 0008 train_loss= 1.34533 time= 0.07168
Epoch: 0009 train_loss= 1.32329 time= 0.07389
Epoch: 0010 train_loss= 1.29378 time= 0.07264
Epoch: 0011 train_loss= 1.25959 time= 0.06860
Epoch: 0012 train_loss= 1.22415 time= 0.07627
Epoch: 0013 train_loss= 1.19641 time= 0.07595
Epoch: 0014 train_loss= 1

Epoch: 0174 train_loss= 0.67121 time= 0.07055
Epoch: 0175 train_loss= 0.68251 time= 0.07491
Epoch: 0176 train_loss= 0.70411 time= 0.07300
Epoch: 0177 train_loss= 0.66351 time= 0.06895
Epoch: 0178 train_loss= 0.66993 time= 0.07277
Epoch: 0179 train_loss= 0.67770 time= 0.07102
Epoch: 0180 train_loss= 0.66143 time= 0.08064
Epoch: 0181 train_loss= 0.65033 time= 0.07895
Epoch: 0182 train_loss= 0.68383 time= 0.08400
Epoch: 0183 train_loss= 0.64692 time= 0.07700
Epoch: 0184 train_loss= 0.65569 time= 0.08000
Epoch: 0185 train_loss= 0.67299 time= 0.07810
Epoch: 0186 train_loss= 0.63273 time= 0.07705
Epoch: 0187 train_loss= 0.65310 time= 0.07799
Epoch: 0188 train_loss= 0.66839 time= 0.07800
Epoch: 0189 train_loss= 0.62496 time= 0.07695
Epoch: 0190 train_loss= 0.65304 time= 0.07742
Epoch: 0191 train_loss= 0.66694 time= 0.07200
Epoch: 0192 train_loss= 0.61905 time= 0.06940
Epoch: 0193 train_loss= 0.68895 time= 0.08168
Epoch: 0194 train_loss= 0.67632 time= 0.06645
Epoch: 0195 train_loss= 0.62132 ti

Epoch: 0152 train_loss= 0.63516 time= 0.07815
Epoch: 0153 train_loss= 0.62845 time= 0.07680
Epoch: 0154 train_loss= 0.64902 time= 0.08003
Epoch: 0155 train_loss= 0.63893 time= 0.07902
Epoch: 0156 train_loss= 0.63248 time= 0.07800
Epoch: 0157 train_loss= 0.62846 time= 0.07600
Epoch: 0158 train_loss= 0.62419 time= 0.08100
Epoch: 0159 train_loss= 0.62275 time= 0.08300
Epoch: 0160 train_loss= 0.61867 time= 0.07595
Epoch: 0161 train_loss= 0.61966 time= 0.07000
Epoch: 0162 train_loss= 0.62260 time= 0.07400
Epoch: 0163 train_loss= 0.62412 time= 0.07700
Epoch: 0164 train_loss= 0.65407 time= 0.07800
Epoch: 0165 train_loss= 0.67249 time= 0.07101
Epoch: 0166 train_loss= 0.72499 time= 0.07532
Epoch: 0167 train_loss= 0.63311 time= 0.07704
Epoch: 0168 train_loss= 0.61628 time= 0.07595
Epoch: 0169 train_loss= 0.69152 time= 0.07605
Epoch: 0170 train_loss= 0.63454 time= 0.07798
Epoch: 0171 train_loss= 0.59995 time= 0.07502
Epoch: 0172 train_loss= 0.62841 time= 0.07794
Epoch: 0173 train_loss= 0.62870 ti